# Flowers Image Classification using a Neural Network
In this notebook, we show how to build a neural network to classify the tf-flowers dataset.
Much of the data exploration was done in the companion notebook: 02a_machine_perception.ipynb

In [1]:
# ========================================================
# Step 0: Imports & environment
# ========================================================
from pathlib import Path
import platform
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image, ImageDraw, ImageFont

tf.random.set_seed(42)

# -------------------------------
# Windows TensorFlow stability
# -------------------------------
if platform.system() == "Windows":
    # Disable GPU to prevent native crashes
    try:
        tf.config.set_visible_devices([], 'GPU')
    except:
        pass

    # Limit threading to avoid MKL / OpenMP conflicts
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)

# Output directory for generated images & CSVs
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

print("TensorFlow version:", tf.__version__)
if tf.config.list_physical_devices('GPU'):
    print("GPU devices available:", tf.config.list_physical_devices('GPU'))
else:
    print("No GPU detected, running on CPU.")

# ========================================================
# Step 1: Load TFDS flowers dataset
# ========================================================
(ds_train, ds_val), ds_info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True,
    data_dir=Path("data") / "tfds"
)

CLASS_NAMES = ds_info.features["label"].names
NUM_CLASSES = len(CLASS_NAMES)

print(f"Classes ({NUM_CLASSES}): {CLASS_NAMES}")
print(f"Train samples: {ds_info.splits['train'].num_examples * 0.8:.0f}")
print(f"Validation samples: {ds_info.splits['train'].num_examples * 0.2:.0f}")


TensorFlow version: 2.9.1
No GPU detected, running on CPU.
Classes (5): ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
Train samples: 2936
Validation samples: 734


In [2]:
# ========================================================
# Step 2: Simple dataset preprocessing
# ========================================================
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3
BATCH_SIZE = 16

def preprocess(img, label):
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

train_dataset = ds_train.map(preprocess).batch(BATCH_SIZE)
val_dataset = ds_val.map(preprocess).batch(BATCH_SIZE)

# ========================================================
# Step 3: Define simple neural network
# ========================================================
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 5)                 752645    
                                                                 
Total params: 752,645
Trainable params: 752,645
Non-trainable params: 0
_________________________________________________________________


In [3]:
# ========================================================
# Step 4: Train the model
# ========================================================
EPOCHS = 5  # adjust for demo speed
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)


Epoch 1/5
184/184 [==============================] - 3s 14ms/step - loss: 18.3946 - accuracy: 0.3283 - val_loss: 17.5353 - val_accuracy: 0.3624
Epoch 2/5
184/184 [==============================] - 2s 8ms/step - loss: 10.2625 - accuracy: 0.4251 - val_loss: 9.1255 - val_accuracy: 0.4033
Epoch 3/5
184/184 [==============================] - 2s 8ms/step - loss: 8.8731 - accuracy: 0.4469 - val_loss: 12.2511 - val_accuracy: 0.3815
Epoch 4/5
184/184 [==============================] - 2s 8ms/step - loss: 12.3787 - accuracy: 0.4469 - val_loss: 13.5211 - val_accuracy: 0.4210
Epoch 5/5
184/184 [==============================] - 2s 8ms/step - loss: 9.4573 - accuracy: 0.5041 - val_loss: 9.8265 - val_accuracy: 0.4101


In [4]:
# ========================================================
# Step 5: Export training metrics to CSV
# ========================================================
import csv

metrics_csv = OUT_DIR / "training_metrics.csv"
fieldnames = ['epoch'] + list(history.history.keys())

with open(metrics_csv, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(EPOCHS):
        row = {'epoch': i+1}
        for key in history.history:
            row[key] = history.history[key][i]
        writer.writerow(row)

print(f"Training metrics saved to {metrics_csv.resolve()}")


Training metrics saved to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs\training_metrics.csv


In [5]:
# ========================================================
# Step 6: Visualize predictions (PIL grid)
# ========================================================
def save_prediction_grid(dataset, model, out_path, num_images=15, grid_shape=(3,5)):
    imgs, labels, preds, probs = [], [], [], []
    
    for idx, (img, label) in enumerate(dataset.unbatch().take(num_images)):
        batch_img = tf.expand_dims(img, 0)
        pred = model.predict(batch_img, verbose=0)[0]
        pred_idx = int(tf.argmax(pred))
        imgs.append((img.numpy()*255).astype(np.uint8))
        labels.append(int(label))
        preds.append(pred_idx)
        probs.append(pred[pred_idx])
    
    # PIL image grid
    rows, cols = grid_shape
    thumb_h, thumb_w = IMG_HEIGHT, IMG_WIDTH
    grid_img = Image.new("RGB", (cols*thumb_w, rows*thumb_h), "white")
    draw = ImageDraw.Draw(grid_img)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()
    
    for i in range(len(imgs)):
        r, c = divmod(i, cols)
        img = Image.fromarray(imgs[i])
        grid_img.paste(img, (c*thumb_w, r*thumb_h))
        text = f"{CLASS_NAMES[labels[i]]} -> {CLASS_NAMES[preds[i]]} ({probs[i]:.2f})"
        draw.text((c*thumb_w + 5, r*thumb_h + 5), text, fill="black", font=font)
    
    grid_img.save(out_path)
    print(f"Prediction grid saved to {out_path.resolve()}")

save_prediction_grid(val_dataset, model, OUT_DIR / "prediction_grid.png")


Prediction grid saved to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs\prediction_grid.png


In [6]:
# ========================================================
# Step 7: Visualize linear layer weights (PIL)
# ========================================================
def save_trained_weights(model, out_dir):
    weights = model.layers[1].get_weights()[0]  # Dense layer weights
    min_wt = np.min(weights)
    max_wt = np.max(weights)
    scaled_weights = (weights - min_wt) / (max_wt - min_wt)
    
    for i in range(NUM_CLASSES):
        w_img = (scaled_weights[:, i].reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS) * 255).astype(np.uint8)
        img = Image.fromarray(w_img)
        img.save(out_dir / f"weights_{CLASS_NAMES[i]}.png")
        print(f"Saved weight visualization for {CLASS_NAMES[i]}")

save_trained_weights(model, OUT_DIR)


Saved weight visualization for dandelion
Saved weight visualization for daisy
Saved weight visualization for tulips
Saved weight visualization for sunflowers
Saved weight visualization for roses


## Now, let's go more in depth
### Setup: Install Keras Tuner: `conda install -c conda-forge keras-tuner`

In [7]:
# ========================================================
# Imports
# ========================================================
from pathlib import Path
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import csv
import keras_tuner as kt

tf.random.set_seed(42)
OUT_DIR = Path("outputs_nn")
OUT_DIR.mkdir(exist_ok=True)

IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3
CLASS_NAMES = None

# ========================================================
# Step 1: Load TFDS flowers dataset
# ========================================================
(ds_train, ds_val), ds_info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True,
    data_dir=Path("data") / "tfds"
)

CLASS_NAMES = ds_info.features["label"].names
NUM_CLASSES = len(CLASS_NAMES)
print("Classes:", CLASS_NAMES)


Classes: ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']


In [8]:
# ========================================================
# Step 2: Preprocess datasets
# ========================================================
BATCH_SIZE = 32
def preprocess(img, label):
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

train_dataset = ds_train.map(preprocess).batch(BATCH_SIZE)
val_dataset = ds_val.map(preprocess).batch(BATCH_SIZE)

# ========================================================
# Step 3: Parameterized NN trainer
# ========================================================
def train_nn_model(train_ds, val_ds, num_hidden=128, lrate=0.001, l1=0.0, l2=0.0, epochs=5):
    reg = tf.keras.regularizers.l1_l2(l1, l2)
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        tf.keras.layers.Dense(num_hidden, activation='relu', kernel_regularizer=reg),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=reg)
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    
    # Save metrics CSV
    metrics_csv = OUT_DIR / f"metrics_hidden{num_hidden}_l1{l1}_l2{l2}.csv"
    fieldnames = ['epoch'] + list(history.history.keys())
    with open(metrics_csv, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(epochs):
            row = {'epoch': i+1}
            for key in history.history:
                row[key] = history.history[key][i]
            writer.writerow(row)
    print(f"Training metrics saved to {metrics_csv.resolve()}")
    return model, history


In [9]:
# ========================================================
# Step 4: Train examples
# ========================================================
model1, _ = train_nn_model(train_dataset, val_dataset, num_hidden=128, lrate=0.0001)

Epoch 1/5
92/92 [==============================] - 7s 70ms/step - loss: 2.2904 - accuracy: 0.3569 - val_loss: 1.4229 - val_accuracy: 0.4251
Epoch 2/5
92/92 [==============================] - 6s 63ms/step - loss: 1.3279 - accuracy: 0.4571 - val_loss: 1.3820 - val_accuracy: 0.4305
Epoch 3/5
92/92 [==============================] - 6s 66ms/step - loss: 1.2537 - accuracy: 0.4826 - val_loss: 1.4216 - val_accuracy: 0.4292
Epoch 4/5
92/92 [==============================] - 6s 68ms/step - loss: 1.2016 - accuracy: 0.5089 - val_loss: 1.4436 - val_accuracy: 0.4305
Epoch 5/5
92/92 [==============================] - 6s 68ms/step - loss: 1.1609 - accuracy: 0.5307 - val_loss: 1.4624 - val_accuracy: 0.4482
Training metrics saved to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs_nn\metrics_hidden128_l10.0_l20.0.csv


In [10]:
model2, _ = train_nn_model(train_dataset, val_dataset, num_hidden=256, lrate=0.0001)

Epoch 1/5
92/92 [==============================] - 11s 118ms/step - loss: 2.9730 - accuracy: 0.3341 - val_loss: 1.5295 - val_accuracy: 0.4101
Epoch 2/5
92/92 [==============================] - 12s 125ms/step - loss: 1.4875 - accuracy: 0.4288 - val_loss: 1.4068 - val_accuracy: 0.4632
Epoch 3/5
92/92 [==============================] - 11s 124ms/step - loss: 1.3712 - accuracy: 0.4547 - val_loss: 1.3991 - val_accuracy: 0.4578
Epoch 4/5
92/92 [==============================] - 12s 125ms/step - loss: 1.3074 - accuracy: 0.4850 - val_loss: 1.3787 - val_accuracy: 0.4591
Epoch 5/5
92/92 [==============================] - 12s 126ms/step - loss: 1.2448 - accuracy: 0.5112 - val_loss: 1.3063 - val_accuracy: 0.4823
Training metrics saved to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs_nn\metrics_hidden256_l10.0_l20.0.csv


In [11]:
model3, _ = train_nn_model(train_dataset, val_dataset, num_hidden=128, lrate=0.0001, l2=0.001)

Epoch 1/5
92/92 [==============================] - 11s 120ms/step - loss: 2.7771 - accuracy: 0.3062 - val_loss: 1.8763 - val_accuracy: 0.3079
Epoch 2/5
92/92 [==============================] - 12s 126ms/step - loss: 1.6801 - accuracy: 0.3900 - val_loss: 1.7608 - val_accuracy: 0.3501
Epoch 3/5
92/92 [==============================] - 12s 128ms/step - loss: 1.5768 - accuracy: 0.4547 - val_loss: 1.7842 - val_accuracy: 0.3774
Epoch 4/5
92/92 [==============================] - 12s 127ms/step - loss: 1.5227 - accuracy: 0.4598 - val_loss: 1.6926 - val_accuracy: 0.4019
Epoch 5/5
92/92 [==============================] - 12s 130ms/step - loss: 1.4967 - accuracy: 0.4768 - val_loss: 1.6218 - val_accuracy: 0.4019
Training metrics saved to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs_nn\metrics_hidden128_l10.0_l20.001.csv


In [12]:
# ========================================================
# Step 5: Prediction grid (PIL)
# ========================================================
def save_prediction_grid(dataset, model, out_path, num_images=15, grid_shape=(3,5)):
    imgs, labels, preds, probs = [], [], [], []
    for idx, (img, label) in enumerate(dataset.unbatch().take(num_images)):
        batch_img = tf.expand_dims(img, 0)
        pred = model.predict(batch_img, verbose=0)[0]
        pred_idx = int(tf.argmax(pred))
        imgs.append((img.numpy()*255).astype(np.uint8))
        labels.append(int(label))
        preds.append(pred_idx)
        probs.append(pred[pred_idx])
    rows, cols = grid_shape
    grid_img = Image.new("RGB", (cols*IMG_WIDTH, rows*IMG_HEIGHT), "white")
    draw = ImageDraw.Draw(grid_img)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()
    for i in range(len(imgs)):
        r, c = divmod(i, cols)
        img = Image.fromarray(imgs[i])
        grid_img.paste(img, (c*IMG_WIDTH, r*IMG_HEIGHT))
        text = f"{CLASS_NAMES[labels[i]]} -> {CLASS_NAMES[preds[i]]} ({probs[i]:.2f})"
        draw.text((c*IMG_WIDTH + 5, r*IMG_HEIGHT + 5), text, fill="black", font=font)
    grid_img.save(out_path)
    print(f"Prediction grid saved to {out_path.resolve()}")

save_prediction_grid(val_dataset, model3, OUT_DIR / "prediction_grid_nn.png")


Prediction grid saved to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs_nn\prediction_grid_nn.png


In [13]:
# ========================================================
# Step 6: Linear layer weight visualization
# ========================================================
def save_trained_weights(model, out_dir):
    weights = model.layers[1].get_weights()[0]  # Dense layer weights
    min_wt = np.min(weights)
    max_wt = np.max(weights)
    scaled_weights = (weights - min_wt) / (max_wt - min_wt)
    for i in range(NUM_CLASSES):
        w_img = (scaled_weights[:, i].reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS) * 255).astype(np.uint8)
        img = Image.fromarray(w_img)
        img.save(out_dir / f"weights_hidden{i}.png")
        print(f"Saved weight visualization for {CLASS_NAMES[i]}")

save_trained_weights(model3, OUT_DIR)


Saved weight visualization for dandelion
Saved weight visualization for daisy
Saved weight visualization for tulips
Saved weight visualization for sunflowers
Saved weight visualization for roses


## Hyperparameter Tuning

In [14]:
# ========================================================
# Optional: Hyperparameter Tuning Demo (CPU-friendly)
# ========================================================

# Take a small subset for demonstration
train_subset = train_dataset.take(200)
val_subset = val_dataset.take(50)

def build_tuned_model(hp):
    num_hidden = hp.Int('num_hidden', 32, 256, step=32)
    l2 = hp.Choice('l2', values=[0.0, 0.001, 0.01])
    lrate = hp.Float('lrate', 1e-4, 1e-2, sampling='log')
    reg = tf.keras.regularizers.l2(l2)

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        tf.keras.layers.Dense(num_hidden, activation='relu', kernel_regularizer=reg),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=reg)
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.BayesianOptimization(
    build_tuned_model,
    objective='val_accuracy',
    max_trials=3,  # keep low for CPU demo
    num_initial_points=1,
    overwrite=True
)

tuner.search(train_subset, validation_data=val_subset, epochs=3)

# Get top hyperparameters and model
best_hp = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]
print("Best hyperparameters:", best_hp.values)
best_model.summary()


Trial 3 Complete [00h 01m 05s]
val_accuracy: 0.42643052339553833

Best val_accuracy So Far: 0.4564032554626465
Total elapsed time: 00h 02m 24s
Best hyperparameters: {'num_hidden': 256, 'l2': 0.0, 'lrate': 0.0001}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 256)               38535424  
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 38,536,709
Trainable params: 38,536,709
Non-trainable params: 0
_________________________________________________________________


## Deep Neural Network
### Setup: Install Pandas: `conda install -c conda-forge pandas`
Let's train a DNN. We will parameterize the number of layers, and the number of nodes in each layer

In [17]:
# ========================================================
# Deep Neural Network (DNN) example - CPU-friendly, TFDS
# ========================================================
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3

# Use the same TFDS flowers dataset from previous notebook
def preprocess(img, label):
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.cast(img, tf.float32)
    return img, label

train_dataset = ds_train.map(preprocess).batch(BATCH_SIZE)
eval_dataset = ds_val.map(preprocess).batch(BATCH_SIZE)

# ========================================================
# Function to build/train DNN with arbitrary hidden layers
# ========================================================
def train_and_evaluate_dnn(batch_size=32,
                           lrate=0.0001,
                           l1=0,
                           l2=0.001,
                           dropout_prob=0.4,
                           num_hidden=[64, 16],
                           epochs=10):
    regularizer = tf.keras.regularizers.l1_l2(l1=l1, l2=l2)
    
    # Build sequential model
    layers = [tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                                      name='input_pixels')]
    
    # Add hidden layers with BatchNorm, ReLU, and Dropout
    for hno, nodes in enumerate(num_hidden):
        layers.extend([
            tf.keras.layers.Dense(nodes, kernel_regularizer=regularizer,
                                  name=f'hidden_dense_{hno}'),
            tf.keras.layers.BatchNormalization(scale=False, center=False,
                                               name=f'batchnorm_dense_{hno}'),
            tf.keras.layers.Activation('relu', name=f'relu_dense_{hno}'),
            tf.keras.layers.Dropout(rate=dropout_prob, name=f'dropout_dense_{hno}')
        ])
    
    # Output layer
    layers.append(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', 
                                        kernel_regularizer=regularizer,
                                        name='flower_prob'))
    
    model = tf.keras.Sequential(layers, name='flower_classification')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    # Train model
    history = model.fit(train_dataset, validation_data=eval_dataset, epochs=epochs)
    
    # Export CSV for plotting outside matplotlib
    import pandas as pd
    df = pd.DataFrame(history.history)
    df.to_csv("dnn_training_history.csv", index=False)
    
    # Print some key metrics in terminal for quick inspection
    print("\nFinal training metrics:")
    for key, value in history.history.items():
        print(f"{key}: {value[-1]:.4f}")
    
    return model

# ========================================================
# Train a simple DNN with 2 hidden layers
# ========================================================
model = train_and_evaluate_dnn(num_hidden=[64, 16], dropout_prob=0.4)


Epoch 1/10
92/92 [==============================] - 7s 67ms/step - loss: 1.8526 - accuracy: 0.3028 - val_loss: 2.0764 - val_accuracy: 0.3420
Epoch 2/10
92/92 [==============================] - 6s 64ms/step - loss: 1.6986 - accuracy: 0.3386 - val_loss: 1.5611 - val_accuracy: 0.4046
Epoch 3/10
92/92 [==============================] - 6s 67ms/step - loss: 1.6362 - accuracy: 0.3808 - val_loss: 1.5855 - val_accuracy: 0.3842
Epoch 4/10
92/92 [==============================] - 6s 69ms/step - loss: 1.5831 - accuracy: 0.3910 - val_loss: 1.5038 - val_accuracy: 0.4768
Epoch 5/10
92/92 [==============================] - 6s 69ms/step - loss: 1.5521 - accuracy: 0.4063 - val_loss: 1.4514 - val_accuracy: 0.4632
Epoch 6/10
92/92 [==============================] - 6s 69ms/step - loss: 1.5135 - accuracy: 0.4292 - val_loss: 1.6235 - val_accuracy: 0.3774
Epoch 7/10
92/92 [==============================] - 6s 69ms/step - loss: 1.4737 - accuracy: 0.4380 - val_loss: 1.4608 - val_accuracy: 0.4591
Epoch 8/10
92

## Diagrams (Key Concepts)

In [18]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Create a subfolder for diagrams
DIAGRAM_DIR = "diagrams"
os.makedirs(DIAGRAM_DIR, exist_ok=True)

# ===============================
# Activation Functions Diagrams
# ===============================

x = np.arange(-10.0, 10.0, 0.1)

activations = {
    'sigmoid': {
        'func': tf.keras.activations.sigmoid,
        'desc': ("Sigmoid squashes inputs to [0,1]; extreme negatives -> 0, positives -> 1. "
                 "Good for probabilities but can cause vanishing gradients.")
    },
    'relu': {
        'func': tf.keras.activations.relu,
        'desc': ("ReLU outputs 0 for negatives, linear for positives. "
                 "Common in hidden layers, avoids vanishing gradients.")
    },
    'elu': {
        'func': tf.keras.activations.elu,
        'desc': ("ELU is smooth for negatives, linear for positives; improves learning speed/stability.")
    }
}

for name, info in activations.items():
    y = info['func'](x).numpy()
    y_min, y_max = np.min(y), np.max(y)
    y_scaled = ((y - y_min) / (y_max - y_min) * 255).astype(np.uint8)
    
    width, height = 500, 300
    img = Image.new("RGB", (width, height), color="white")
    draw = ImageDraw.Draw(img)
    
    # Draw axes
    draw.line((50, 0, 50, height), fill="black")      # y-axis
    draw.line((0, height-50, width, height-50), fill="black")  # x-axis
    
    # Draw curve
    for i in range(len(x)-1):
        x0 = int(i / len(x) * (width-60)) + 50
        y0 = height-50 - int(y_scaled[i] / 255 * (height-60))
        x1 = int((i+1) / len(x) * (width-60)) + 50
        y1 = height-50 - int(y_scaled[i+1] / 255 * (height-60))
        draw.line((x0, y0, x1, y1), fill="blue", width=2)
    
    # Title and description
    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except:
        font = ImageFont.load_default()
    draw.text((10, 10), f"{name} activation", fill="black", font=font)
    draw.text((10, height-45), info['desc'], fill="black", font=font)
    
    # Save PNG in subfolder
    filepath = os.path.join(DIAGRAM_DIR, f"{name}_activation.png")
    img.save(filepath)
    print(f"Saved {filepath}")

# ===============================
# Linear vs Small Deep Model
# ===============================

IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3
num_classes = 5

linear_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
], name='linear_model')

deep_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
], name='small_deep_model')

def export_model_summary_png(model, filename):
    summary_lines = []
    model.summary(print_fn=lambda x: summary_lines.append(x))
    summary_text = "\n".join(summary_lines)
    
    img = Image.new("RGB", (700, 200), color="white")
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 12)
    except:
        font = ImageFont.load_default()
    
    y_text = 5
    for line in summary_text.split("\n"):
        draw.text((5, y_text), line, fill="black", font=font)
        y_text += 14

    filepath = os.path.join(DIAGRAM_DIR, filename)
    img.save(filepath)
    print(f"Saved {filepath}")

export_model_summary_png(linear_model, "linear_model_summary.png")
export_model_summary_png(deep_model, "small_deep_model_summary.png")


Saved diagrams\sigmoid_activation.png
Saved diagrams\relu_activation.png
Saved diagrams\elu_activation.png
Saved diagrams\linear_model_summary.png
Saved diagrams\small_deep_model_summary.png
